In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections

# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold

In [67]:
df=pd.read_csv("returns_data.csv")
df1=pd.read_csv("returns_test.csv")
df.head()
#df1.head()

,time_of_purchase,club_membership,checkout_type,payment_method,product_code,department,items_in_purchase,cost,cashier,return
0,21062021--10:51,pos,regular,3210slv_585,NIK3_dscnt_3906786,grocery,10,25.00,45,0
1,21062021--11:04,pos,regular,1680slv_162,NIK3_dscnt_3710510,fashion,126,291.60,48,1
2,21062021--11:16,neg,regular,5630gld_656,CONV_resp_2019589,ready food,13,1.28,69,0
3,21062021--11:16,neg,regular,1680std_266,CONV_resp_4862207,small appliances,9,13.93,31,0
4,21062021--11:17,neg,regular,4580std_304,CONV_ssnl_2433974,hobby,14,11.33,51,0


In [68]:
#Change the coloumn club_membership to nomeric value  
df.loc[df['club_membership']=='pos','club_membership']=0
df.loc[df['club_membership']=='neg','club_membership']=1

#Change the coloumn checkout_type  to nomeric value
df.loc[df['checkout_type']=='regular','checkout_type']=0
df.loc[df['checkout_type']=='self checkout','checkout_type']=1

# Leaves in column payment_method only the middle and change it to nomeric value 
df['payment_method'] = df.apply(lambda x: x['payment_method'][:-4], axis = 1)
df['payment_method'] = df.apply(lambda x: x['payment_method'][4:], axis = 1)
df.loc[df['payment_method']=='slv','payment_method']=0
df.loc[df['payment_method']=='gld','payment_method']=1
df.loc[df['payment_method']=='std','payment_method']=2
df.loc[df['payment_method']=='plt','payment_method']=3

# Leaves in column product_code only the middle and change it to nomeric value 
a=df['product_code'].str.split('_',expand=True)
a.drop(a.columns[[0]],axis=1,inplace=True)
a.drop(a.columns[[1]],axis=1,inplace=True)
df['product_code']=a
df.loc[df['product_code']=='dscnt','product_code']=0
df.loc[df['product_code']=='resp','product_code']=1
df.loc[df['product_code']=='ssnl','product_code']=2
df.loc[df['product_code']=='spcl','product_code']=3
df.loc[df['product_code']=='grn4s','product_code']=4
df.loc[df['product_code']=='frstn','product_code']=5
df.loc[df['product_code']=='rmvb','product_code']=6
df.loc[df['product_code']=='updt','product_code']=7
df.loc[df['product_code']=='scrb','product_code']=8
df.loc[df['product_code']=='rnwd','product_code']=9
df['product_code'].unique()
#change the column department to nomeric value:
combine = [df]
opinion_mapping = {"grocery": 0, "fashion": 1, "ready food": 3,'small appliances':4,'hobby':5,'books':6,
'toys':7,'sports':8,'beauty':9,'office':10,'mobile':11,'furniture':12,'garden':13,
'auto':14,'tools':15,'stationery':16,'computers':17,'leisure':18,'large appliances':19,
'baby':20,'electronics':21,'lighting':21,'photography':22,'professional':23,'entertainment':24,
'luggage':25,'games':26,'special orders':27,'pet supplis':28,'decorative':29,
'arts':30,'optics':31,'outdoor':32,'organization':33,'security':34,'music':35,
'beddng':36,'medical':37,'outdoor furniture':38, 'software':39,'plumbing':40,'houseware':41}

for i in combine:
    i['department'] = i['department'].map(opinion_mapping)
    i['department'] = i['department'].fillna(0)

#Change the type of the colomns to integer
def convert_dtype(df,c):
    df[c]=pd.to_numeric(df[c],errors='coerce')
c=['club_membership','checkout_type','payment_method','product_code']

for c in c:
    convert_dtype(df,c)

    
#Same for the df1
#Change the coloumn club_membership to nomeric value  
df1.loc[df1['club_membership']=='pos','club_membership']=0
df1.loc[df1['club_membership']=='neg','club_membership']=1
#Change the coloumn checkout_type  to nomeric value
df1.loc[df1['checkout_type']=='regular','checkout_type']=0
df1.loc[df1['checkout_type']=='self checkout','checkout_type']=1
# Leaves in column payment_method only the middle and change it to nomeric value 
df1['payment_method'] = df1.apply(lambda x: x['payment_method'][:-4], axis = 1)
df1['payment_method'] = df1.apply(lambda x: x['payment_method'][4:], axis = 1)
df1.loc[df1['payment_method']=='slv','payment_method']=0
df1.loc[df1['payment_method']=='gld','payment_method']=1
df1.loc[df1['payment_method']=='std','payment_method']=2
df1.loc[df1['payment_method']=='plt','payment_method']=3
# Leaves in column product_code only the middle and change it to nomeric value 
a=df1['product_code'].str.split('_',expand=True)
a.drop(a.columns[[0]],axis=1,inplace=True)
a.drop(a.columns[[1]],axis=1,inplace=True)
df1['product_code']=a
df1.loc[df1['product_code']=='dscnt','product_code']=0
df1.loc[df1['product_code']=='resp','product_code']=1
df1.loc[df1['product_code']=='ssnl','product_code']=2
df1.loc[df1['product_code']=='spcl','product_code']=3
df1.loc[df1['product_code']=='grn4s','product_code']=4
df1.loc[df1['product_code']=='frstn','product_code']=5
df1.loc[df1['product_code']=='rmvb','product_code']=6
df1.loc[df1['product_code']=='updt','product_code']=7
df1.loc[df1['product_code']=='scrb','product_code']=8
df1.loc[df1['product_code']=='rnwd','product_code']=9
df1['product_code'].unique()
#change the column department to nomeric value:
combine1 = [df1]
opinion_mapping = {"grocery": 0, "fashion": 1, "ready food": 3,'small appliances':4,'hobby':5,'books':6,
'toys':7,'sports':8,'beauty':9,'office':10,'mobile':11,'furniture':12,'garden':13,
'auto':14,'tools':15,'stationery':16,'computers':17,'leisure':18,'large appliances':19,
'baby':20,'electronics':21,'lighting':21,'photography':22,'professional':23,'entertainment':24,
'luggage':25,'games':26,'special orders':27,'pet supplis':28,'decorative':29,
'arts':30,'optics':31,'outdoor':32,'organization':33,'security':34,'music':35,
'beddng':36,'medical':37,'outdoor furniture':38, 'software':39,'plumbing':40,'houseware':41}

for i in combine1:
    i['department'] = i['department'].map(opinion_mapping)
    i['department'] = i['department'].fillna(0)

#Change the type of the colomns
def convert_dtype(df1,c):
    df1[c]=pd.to_numeric(df1[c],errors='coerce')
c=['club_membership','checkout_type','payment_method','product_code']

for c in c:
    convert_dtype(df1,c)
    
    

In [70]:
df.head()
#df1.head()

,time_of_purchase,club_membership,checkout_type,payment_method,product_code,department,items_in_purchase,cost,cashier,return
0,28062021--10:21,1,0,3,1,3,2,8.63,88,NaN
1,28062021--11:03,1,0,3,1,41,30,33.06,45,NaN
2,28062021--11:06,1,0,2,4,0,9,1270.21,49,NaN
3,28062021--11:16,1,0,3,4,12,9,882.09,31,NaN
4,28062021--11:24,1,0,1,4,7,93,564.54,44,NaN


In [89]:
df.dtypes

time_of_purchase      object
club_membership        int64
checkout_type          int64
payment_method         int64
product_code           int64
department             int64
items_in_purchase      int64
cost                 float64
cashier                int64
return                 int64
dtype: object

In [71]:
x=df[['club_membership','checkout_type','payment_method','product_code','department','items_in_purchase','cost','cashier']]
y=df['return']
x.head()

,club_membership,checkout_type,payment_method,product_code,department,items_in_purchase,cost,cashier
0,0,0,0,0,0,10,25.00,45
1,0,0,0,0,1,126,291.60,48
2,1,0,1,1,3,13,1.28,69
3,1,0,2,1,4,9,13.93,31
4,1,0,2,2,5,14,11.33,51


In [72]:
Xnew = df1[['club_membership','checkout_type','payment_method','product_code','department','items_in_purchase','cost','cashier']]
ynew =df1['return']


In [73]:
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier":RandomForestClassifier()
}

In [74]:
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(x,y)
    training_score = cross_val_score(classifier,x,y, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

C:\Users\mby77\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\mby77\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

Classifiers:  LogisticRegression Has a training score of 93.0 % accuracy score
Classifiers:  KNeighborsClassifier Has a training score of 93.0 % accuracy score
Classifiers:  SVC Has a training score of 91.0 % accuracy score
Classifiers:  DecisionTreeClassifier Has a training score of 96.0 % accuracy score
Classifiers:  RandomForestClassifier Has a training score of 97.0 % accuracy score


In [75]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(x, y)
ypred = model.predict(Xnew)
ypred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [76]:
importances = model.feature_importances_
importances*100

array([ 6.23134824,  0.34670167,  2.00569787, 27.50357716, 14.33262366,
       19.71545701, 23.30536039,  6.559234  ])

In [77]:
# I took out three columns with a low rating
#'club_membership','checkout_type' ,'payment_method'

In [78]:
xf=df[['product_code','department','items_in_purchase','cost','cashier']]
yf=df['return']
Xnewf = df1[['product_code','department','items_in_purchase','cost','cashier']]
ynewf =df1['return']
model_new = RandomForestClassifier(n_estimators=1000)
model.fit(xf, yf)
ypred_newf = model.predict(Xnewf)
ypred_newf

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [80]:
model.score(xf,yf)

0.9998181818181818

In [ ]:
df1[return]=ypred_newf